<a href="https://colab.research.google.com/github/Perwil/Trading-Project-California-State/blob/main/Copy_of_Simple_GROUNDWATER_PROBLEM_CALIFORNIA_STATE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import cvxpy as cp

In [ ]:
!git clone https://github.com/cvxpy/cvxpy.git

Cloning into 'cvxpy'...
remote: Enumerating objects: 82586, done.
remote: Counting objects: 100% (3005/3005), done.
remote: Compressing objects: 100% (789/789), done.
remote: Total 82586 (delta 2249), reused 2736 (delta 2073), pack-reused 79581
Receiving objects: 100% (82586/82586), 198.20 MiB | 8.02 MiB/s, done.
Resolving deltas: 100% (58639/58639), done.


In [ ]:
def cas(array_str):
    # Remove the brackets and split by space
    array_elements = "array_str".strip('[]').split()
    # Convert the elements to integers
    array_elements = [int(element) for element in array_elements]
    return array_elements

In [ ]:
def get_array_of_size_k(k):
    while True:
        user_input = input(f"Please enter {k} numbers separated by spaces: ")
        try:
            array = list(map(float, user_input.split()))
            if len(array) == k:
                return array
            else:
                print(f"Error: You must enter exactly {k} numbers.")
        except ValueError:
            print("Error: Please enter valid numbers.")

In [ ]:
def get_array_of_size_n_identical_or_different(n):
    #n = int(input("Enter the size of the array: "))
    identical = input("Do you want all elements to be identical? (True/False): ").strip().lower()

    if identical == "true":
        element = float(input(f"Enter the identical element to fill the array of size {n}: "))
        array = [element] * n
    elif identical == "false":
        while True:
            user_input = input(f"Enter {n} different numbers separated by spaces: ")
            array = list(map(float, user_input.split()))
            if len(array) == n and len(set(array)) == n:
                break
            else:
                print(f"Error: You must enter exactly {n} different numbers.")
    else:
        print("Invalid input. Please enter 'True' or 'False'.")
        return get_array_of_size_n_identical_or_different()

    return array

In [ ]:
class Groundwater():
  def get_array_of_size_k(k):
    while True:
        user_input = input(f"Please enter {k} numbers separated by spaces: ")
        try:
            array = list(map(float, user_input.split()))
            if len(array) == k:
                return array
            else:
                print(f"Error: You must enter exactly {k} numbers.")
        except ValueError:
            print("Error: Please enter valid numbers.")
  def __init__(self, Number_Agents, Types_Products, Profit_Agent, Amount_Water_acre_feet, Maximum_Water_Consumption_Y_j, Phi_Agent_2, Phi_Agent_3, v):
    #self.Types_Products=[2, 2, 2]
    self.Types_Products=Types_Products
    #self.Profit_Agent=[np.array([10, 10]), np.array([10, 10]), np.array([10, 10])]
    self.Profit_Agent=Profit_Agent
    #self.Amount_Water_acre_feet=[np.array([5, 5]), np.array([5, 5]), np.array([5, 5])]
    self.Amount_Water_acre_feet=Amount_Water_acre_feet
    self.Maximum_Water_Consumption_Y_j=Maximum_Water_Consumption_Y_j
    #self.Profit_Agent=[get_array_of_size_k(i) for i in Types_Products]
    #self.Amount_Water_acre_feet=[get_array_of_size_k(i) for i in Types_Products]
    #self.Maximum_Water_Consumption_Y_j=np.array([100000, 750000, 50000])
    #self.Phi_Agent_2=np.array([8000, 8000])
    self.Phi_Agent_2=Phi_Agent_2
    #self.Phi_Agent_3=np.array([8000, 8000])
    self.Phi_Agent_3=Phi_Agent_3
    self.v=v
  def first_opt(self):
    Phi_Agent_1=cp.Variable(2)
    #self.v = cp.Parameter(nonneg=True)
    #self.v.value = 10
    Constraints=[]
    Constraints+=[self.Amount_Water_acre_feet[0] @ Phi_Agent_1+self.Amount_Water_acre_feet[1] @ self.Phi_Agent_2+self.Amount_Water_acre_feet[2]@ self.Phi_Agent_3==cp.sum(self.Maximum_Water_Consumption_Y_j)]
    Constraints+=[Phi_Agent_1[0]>=0, Phi_Agent_1[1]>=0 ]
    for i in range(self.Types_Products[0]):
      Constraints+=[Phi_Agent_1[i]<=10000000]
    Omega=cp.Parameter(3)
    Omega.value=np.array([1/3, 1/3, 1/3])
    Obj=[self.Profit_Agent[0]@Phi_Agent_1+(self.Maximum_Water_Consumption_Y_j[i]-self.Amount_Water_acre_feet[0]@Phi_Agent_1)*self.v for i in range(3)]
    Obj1=[cp.log(Obj[i]) for i in range(3)]
    Objective=Omega.value @ Obj1
    Problem_to_solve=cp.Problem(cp.Maximize(Objective), Constraints)
    Sol=Problem_to_solve.solve()
    self.result=np.array([Phi_Agent_1.value[0], Phi_Agent_1.value[1]])
    return print("Problem Objective is DCP:", Problem_to_solve.is_dcp()), print('The solution is:', Sol), print("Quantity of Products 1:", self.result)
  def second_opt(self):
    Phi_Agentb_2=cp.Variable(2)
    #self.Phi_Agent_1=self.result
    #self.Phi_Agent_3=Phi_Agent_3
    #self.v = cp.Parameter(nonneg=True)
    #self.v.value = 10
    Constraints=[]
    Constraints+=[self.Amount_Water_acre_feet[0] @ self.result+self.Amount_Water_acre_feet[1] @ Phi_Agentb_2+self.Amount_Water_acre_feet[2] @ self.Phi_Agent_3==cp.sum(self.Maximum_Water_Consumption_Y_j)]
    Constraints+=[Phi_Agentb_2[0]>=0, Phi_Agentb_2[1]>=0 ]
    for i in range(self.Types_Products[0]):
      Constraints+=[Phi_Agentb_2[i]<=10000000]
    Omega=cp.Parameter(3)
    Omega.value=np.array([1/3, 1/3, 1/3])
    Obj=[self.Profit_Agent[1]@Phi_Agentb_2+(self.Maximum_Water_Consumption_Y_j[i]-self.Amount_Water_acre_feet[0]@Phi_Agentb_2)*self.v for i in range(3)]
    Obj1=[cp.log(Obj[i]) for i in range(3)]
    Objective=Omega.value @ Obj1
    Problem_to_solve=cp.Problem(cp.Maximize(Objective), Constraints)
    Sol=Problem_to_solve.solve()
    self.result1=np.array([Phi_Agentb_2.value[0], Phi_Agentb_2.value[1]])
    return print("Problem Objective is DCP:", Problem_to_solve.is_dcp()), print('The solution is:', Sol), print("Quantity of Products 2:", self.result1)
  def Third_opt(self):
    Phi_Agentc_3=cp.Variable(2)
    #self.Phi_Agent_1=self.result
    #self.Phi_Agent_3=Phi_Agent_3
    #self.v = cp.Parameter(nonneg=True)
    #self.v.value = 10
    Constraints=[]
    Constraints+=[self.Amount_Water_acre_feet[0] @ self.result+self.Amount_Water_acre_feet[1] @ self.result1 + self.Amount_Water_acre_feet[2] @ Phi_Agentc_3==cp.sum(self.Maximum_Water_Consumption_Y_j)]
    Constraints+=[Phi_Agentc_3[0]>=0, Phi_Agentc_3[1]>=0 ]
    for i in range(self.Types_Products[0]):
      Constraints+=[Phi_Agentc_3[i]<=10000000]
    Omega=cp.Parameter(3)
    Omega.value=np.array([1/3, 1/3, 1/3])
    Obj=[self.Profit_Agent[2]@Phi_Agentc_3+(self.Maximum_Water_Consumption_Y_j[i]-self.Amount_Water_acre_feet[2]@Phi_Agentc_3)*self.v for i in range(3)]
    Obj1=[cp.log(Obj[i]) for i in range(3)]
    Objective=Omega.value @ Obj1
    Problem_to_solve=cp.Problem(cp.Maximize(Objective), Constraints)
    Sol=Problem_to_solve.solve()
    self.result2=np.array([Phi_Agentc_3.value[0], Phi_Agentc_3.value[1]])
    self.Phi_Agent_2=self.result1
    self.Phi_Agent_3=self.result2
    return print("Problem Objective is DCP:", Problem_to_solve.is_dcp()), print('The solution is:', Sol), print("Quantity of Products 3:", self.result2)


In [ ]:
#Phi_Agent_2=np.array([8000, 8000])
#Phi_Agent_3=np.array([8000, 8000])

In [ ]:
Testit=Groundwater(3, [2, 2, 2], [np.array([50, 50]), np.array([50, 50]), np.array([50, 50])], [np.array([5, 5]), np.array([5, 5]), np.array([5, 5])],
                   np.array([100000, 750000, 50000]), np.array([800, 800]), np.array([8000, 8000]), 10 )

In [ ]:
#Testit=Groundwater(3, [2, 2, 2], [np.array([10, 10]), np.array([10, 10]), np.array([10, 10])], [np.array([5, 5]), np.array([5, 5]), np.array([5, 5])],
                  # np.array([100000, 750000, 50000]), np.array([8000, 8000]), np.array([8000, 8000]), 7 )
for i in range(10):
  Testit.first_opt()
  Testit.second_opt()
  Testit.Third_opt()


Problem Objective is DCP: True
The solution is: 14.256095837958378
Quantity of Products 1: [81199.99855823 81200.00144177]
Problem Objective is DCP: True
The solution is: 14.256095837958378
Quantity of Products 2: [800.00000913 799.99999087]
Problem Objective is DCP: True
The solution is: 14.256095837958378
Quantity of Products 3: [7999.99990701 8000.00009299]
Problem Objective is DCP: True
The solution is: 14.25609583795838
Quantity of Products 1: [81199.99915943 81200.00084057]
Problem Objective is DCP: True
The solution is: 14.256095837958378
Quantity of Products 2: [800.00000914 799.99999086]
Problem Objective is DCP: True
The solution is: 14.256095837958378
Quantity of Products 3: [7999.99990696 8000.00009304]
Problem Objective is DCP: True
The solution is: 14.256095837958378
Quantity of Products 1: [81199.99916813 81200.00083187]
Problem Objective is DCP: True
The solution is: 14.256095837958378
Quantity of Products 2: [799.99999847 800.00000153]
Problem Objective is DCP: True
Th

In [ ]:
#Testit.Third_opt(np.array([17500.00007789, 17499.99992211]), np.array([2499.99999544, 2500.00000456]))

In [ ]:
Testit.first_opt(np.array([17500.00007789, 17499.99992211]), np.array([2499.99999544, 2500.00000456]))

In [ ]:
Testit.second_opt(np.array([2499.99999544, 2500.00000456]))

In [ ]:
def get_array_of_size_k(k):
    while True:
        user_input = input(f"Please enter {k} numbers separated by spaces: ")
        try:
            array = list(map(float, user_input.split()))
            if len(array) == k:
                return array
            else:
                print(f"Error: You must enter exactly {k} numbers.")
        except ValueError:
            print("Error: Please enter valid numbers.")

In [ ]:
L=[3, 2]
print("L")
"print(L)"

L


'print(L)'

In [ ]:
 #Number_Agents = int(input("Please enter a positive integer: "))
 Number_Agents=3

In [ ]:
#Different_types_of_Products_allowed=int(input("Please enter a positive integer: "))
Differents_types_of_Products_per_Agent=2

In [ ]:
#np.random.seed(112)
#Types_Products=get_array_of_size_k(Number_Agents)
#Types_Products=np.array([random.randint(1, Different_types_of_Products_allowed) for i in range(Number_Agents)])

In [ ]:
Types_Products=[2, 2, 2]
# K=[k_1, k_2, k_3,..k_n]=Different Products types produce by Agents

In [ ]:
#List_1=["Profit_Agent_"+str(i) for i in range(Number_Agents)]
#List_1

In [ ]:
def get_array_of_size_k(k):
    while True:
        user_input = input(f"Please enter {k} numbers separated by spaces: ")
        try:
            array = list(map(float, user_input.split()))
            if len(array) == k:
                return array
            else:
                print(f"Error: You must enter exactly {k} numbers.")
        except ValueError:
            print("Error: Please enter valid numbers.")

In [ ]:
#we compute the matrices of Profits of each products of our Agent
#Profit_Agent=dict(zip(List_1, [get_array_of_size_k(i) for i in Types_Products]))
Profit_Agent=[get_array_of_size_k(i) for i in Types_Products]

Please enter 2 numbers separated by spaces: 10 10
Please enter 2 numbers separated by spaces: 10 10
Please enter 2 numbers separated by spaces: 10 10


In [ ]:
Profit_Agent

[[10.0, 10.0], [10.0, 10.0], [10.0, 10.0]]

In [ ]:
#List_2=["Unit_of Good_Water_Consumption_per_Good_Per_Agent_"+str(i) for i in range(Number_Agents)]
#List_2

In [ ]:
#Amount_Water_acre_feet=dict(zip(List_2,[get_array_of_size_k(i) for i in Types_Products]))
#Amount_Water_acre_feet
Amount_Water_acre_feet=[get_array_of_size_k(i) for i in Types_Products]
Amount_Water_acre_feet

Please enter 2 numbers separated by spaces: 5 5
Please enter 2 numbers separated by spaces: 5 5
Please enter 2 numbers separated by spaces: 5 5


[[5.0, 5.0], [5.0, 5.0], [5.0, 5.0]]

In [ ]:
def get_array_of_size_n_identical_or_different(n):
    #n = int(input("Enter the size of the array: "))
    identical = input("Do you want all elements to be identical? (True/False): ").strip().lower()

    if identical == "true":
        element = float(input(f"Enter the identical element to fill the array of size {n}: "))
        array = [element] * n
    elif identical == "false":
        while True:
            user_input = input(f"Enter {n} different numbers separated by spaces: ")
            array = list(map(float, user_input.split()))
            if len(array) == n and len(set(array)) == n:
                break
            else:
                print(f"Error: You must enter exactly {n} different numbers.")
    else:
        print("Invalid input. Please enter 'True' or 'False'.")
        return get_array_of_size_n_identical_or_different()

    return array

# Example usage
#array = get_array_of_size_n_identical_or_different(2)
#print("You entered the array:", array)

In [ ]:
# cost or Price of Extracting one ac-ft of water
#Cost_Extracting_q_j=get_array_of_size_n_identical_or_different(Number_Agents)
#Cost_Extracting_q_j

In [ ]:
# Maximum water consuption Depending on different regime of water High, Medium Low.
Maximum_Water_Consumption_Y_j=get_array_of_size_n_identical_or_different(Number_Agents)
Maximum_Water_Consumption_Y_j

Do you want all elements to be identical? (True/False): false
Enter 3 different numbers separated by spaces: 100000 75000 50000


[100000.0, 75000.0, 50000.0]

In [ ]:
type(Maximum_Water_Consumption_Y_j)

list

In [ ]:
#Production_Variables=[["Phi_"+str(i)+str(j)  for j in range(int(Types_Products[i]))] for i in range(Number_Agents)]
import cvxpy as cp
Phi_Agent_1=cp.Variable(2)
#Phi_Agent_2=cp.Variable(2)
#Phi_Agent_3=cp.Variable(2)
Phi_Agent_2=np.array([2000, 3000])
#Phi_Agent_2[1]=10
Phi_Agent_3=np.array([2000, 3000])
#Phi_Agent_3[1]=15
v = cp.Parameter(nonneg=True)
v.value = 5

In [ ]:
#Production_Variables

In [ ]:
Constraints=[]
Constraints+=[Amount_Water_acre_feet[0]@Phi_Agent_1+Amount_Water_acre_feet[1]@Phi_Agent_2+Amount_Water_acre_feet[2]@Phi_Agent_3==cp.sum(Maximum_Water_Consumption_Y_j)]
Constraints

[Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONNEGATIVE, ()))]

In [ ]:
#import cvxpy as cp
Constraints+=[Phi_Agent_1[0]>=0, Phi_Agent_1[1]>=0 ]
#Constraints+=[Phi_Agent_1[0]>=0, Phi_Agent_1[1]>=0, Phi_Agent_2[0]>=0, Phi_Agent_2[1]>=0, Phi_Agent_3[0]>=0, Phi_Agent_3[1]>=0 ]
for i in range(Types_Products[0]):
  Constraints+=[Phi_Agent_1[i]<=100000000]
 # Constraints+=[Phi_Agent_1[i]<=1000000, Phi_Agent_2[i]<=1000000, Phi_Agent_3[i]<=1000 ]
#Constraints+[Phi_Agent_2[0]==10, Phi_Agent_2[1]==10, Phi_Agent_3[0]==12, Phi_Agent_3[1]==15]
Constraints

[Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Inequality(Constant(CONSTANT, ZERO, ())),
 Inequality(Constant(CONSTANT, ZERO, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ()))]

In [ ]:
Omega=cp.Parameter(3)
Omega.value=np.array([6/10, 3/10, 1/10])
#Types_Products=[int(Types_Products[i]) for i in range(len(Types_Products))]
#for i in range(Number_Agents):
#  for j in range(Types_Products[i]):
#    Production_Variables[i][j]=cp.Variable()
Obj=[Profit_Agent[0]@Phi_Agent_1+(Maximum_Water_Consumption_Y_j[i]-Amount_Water_acre_feet[0]@Phi_Agent_1)*v.value for i in range(3)]
#Obj=[Profit_Agent[0]@Phi_Agent_1-Cost_Extracting_q_j[0]*(Amount_Water_acre_feet[0]@Phi_Agent_1)+(Maximum_Water_Consumption_Y_j[i]-Amount_Water_acre_feet[0]@Phi_Agent_1)*v.value for i in range(3)]

In [ ]:
#len(Obj)
#Omega=np.array([1/3, 1/3, 1/3])

In [ ]:
Obj1=[cp.log(Obj[i]) for i in range(3)]

In [ ]:
Objective=Omega.value @ Obj1
Objective

Expression(CONCAVE, UNKNOWN, ())

In [ ]:
Problem_to_solve=cp.Problem(cp.Maximize(Objective), Constraints)

In [ ]:
print("Problem Objective is DCP:", Problem_to_solve.is_dcp())

Problem Objective is DCP: True


In [ ]:
Sol=Problem_to_solve.solve()
print(Sol)

14.674585075932537


In [ ]:
print(Phi_Agent_1.value)

[17500.00728132 17499.99271868]


In [ ]:
#Amount_Water_acre_feet[0]@Phi_Agent_1.value
#np.dot(Amount_Water_acre_feet[0], Phi_Agent_1.value)
def second_step()

In [ ]:
Phi_Agent_21=cp.Variable(2)
Phi_Agent_31=np.array([2000, 3000])
Maximum_Water_Consumption_Y_j1=np.array([100000.0, 75000.0, 50000.0])

In [ ]:
Constraints_2=[]
Constraints_2+=[Amount_Water_acre_feet[0]@Phi_Agent_1.value + Amount_Water_acre_feet[1]@Phi_Agent_21 + Amount_Water_acre_feet[2]@Phi_Agent_31==cp.sum(Maximum_Water_Consumption_Y_j1)]
Constraints_2+=[Phi_Agent_21[0]>=0, Phi_Agent_21[1]>=0]
for i in range(Types_Products[1]):
  Constraints_2+=[Phi_Agent_21[i]<=100000000]
Constraints_2

[Equality(Expression(AFFINE, UNKNOWN, ()), Expression(CONSTANT, NONNEGATIVE, ())),
 Inequality(Constant(CONSTANT, ZERO, ())),
 Inequality(Constant(CONSTANT, ZERO, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ()))]

In [ ]:
Obj2=[Profit_Agent[1]@Phi_Agent_21+(Maximum_Water_Consumption_Y_j1[i]-Amount_Water_acre_feet[1]@Phi_Agent_21)*v.value for i in range(3)]
Obj2=[cp.log(Obj2[i]) for i in range(3)]
Objective2=Omega.value @ Obj2
Problem_to_solve2=cp.Problem(cp.Maximize(Objective2), Constraints_2)
print("Problem Objective is DCP:", Problem_to_solve2.is_dcp())
Sol2=Problem_to_solve2.solve()
print(Sol2)

Problem Objective is DCP: True
13.46890463172021


In [ ]:
print(Phi_Agent_21.value)

[2500.00074118 2499.99925882]


In [ ]:
Phi_Agent_311=cp.Variable(2)
#Phi_Agent_31=np.array([2000, 3000])
Maximum_Water_Consumption_Y_j11=np.array([100000.0, 75000.0, 50000.0])
Constraints_3=[]
Constraints_3+=[Amount_Water_acre_feet[0]@Phi_Agent_1.value + Amount_Water_acre_feet[1]@Phi_Agent_21.value + Amount_Water_acre_feet[2]@Phi_Agent_311==cp.sum(Maximum_Water_Consumption_Y_j11)]
Constraints_3+=[Phi_Agent_311[0]>=0, Phi_Agent_311[1]>=0]
for i in range(Types_Products[1]):
  Constraints_3+=[Phi_Agent_311[i]<=100000000]
Constraints_3

[Equality(Expression(AFFINE, UNKNOWN, ()), Expression(CONSTANT, NONNEGATIVE, ())),
 Inequality(Constant(CONSTANT, ZERO, ())),
 Inequality(Constant(CONSTANT, ZERO, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ()))]

In [ ]:
Obj3=[Profit_Agent[2]@Phi_Agent_311+(Maximum_Water_Consumption_Y_j11[i]-Amount_Water_acre_feet[2]@Phi_Agent_311)*v.value for i in range(3)]
Obj3=[cp.log(Obj3[i]) for i in range(3)]
Objective3=Omega.value @ Obj3
Problem_to_solve3=cp.Problem(cp.Maximize(Objective3), Constraints_3)
print("Problem Objective is DCP:", Problem_to_solve3.is_dcp())
Sol3=Problem_to_solve3.solve()
print(Sol3)

Problem Objective is DCP: True
13.46890463172021


In [ ]:
print(Phi_Agent_311.value)

[2500.0007361 2499.9992639]


In [ ]:
# I get the keys of the dictionnary Profit_Agent
#keysdict_Profit_Agent=list(Profit_Agent.keys())
#keysdict_Profit_Agent[0]

In [ ]:
#Keysdict_Amount_Water_acre_feet=list(Amount_Water_acre_feet.keys())
#Keysdict_Amount_Water_acre_feet[0]

In [ ]:
#List3=["Function_"+str(i) for i in range(Number_Agents)]
#List3

In [ ]:
#List4=[np.dot(Production_Variables[0],Profit_Agent[keysdict_Profit_Agent[0]])-Cost_Extracting_q_j[0]*np.dot(Production_Variables[0],Amount_Water_acre_feet[Keysdict_Amount_Water_acre_feet[0]])+(Maximum_Water_Consumption-np.dot(Production_Variables[0],Amount_Water_acre_feet[Keysdict_Amount_Water_acre_feet[0]]))*v for Maximum_Water_Consumption in Maximum_Water_Consumption_Y_j]

In [ ]:
#List4=[np.dot(Production_Variables[i],Profit_Agent[keysdict_Profit_Agent[i]])-Cost_Extracting_q_j[i]*np.dot(Production_Variables[i],Amount_Water_acre_feet[Keysdict_Amount_Water_acre_feet[i]])+(Maximum_Water_Consumption_Y_j[i]-np.dot(Production_Variables[i],Amount_Water_acre_feet[Keysdict_Amount_Water_acre_feet[i]]))*v for i in range(Number_Agents)]
#List4

In [ ]:
#Dict_of_Functions=dict(zip(List3, List4))
#Dict_of_Functions

In [ ]:
#for i in range(1, Number_Agents):
 # for j in range(len(Production_Variables[i])):
  #  Production_Variables[i][j]=int(input(f"Please enter a positive integer as initial production quantities phi{i}{j}: "))

In [ ]:
#F_1=np.dot(Production_Variables[0],Profit_Agent['Profit_Agent_0'])-Cost_Extracting_q_j[0]*np.dot(Production_Variables[0],Amount_Water_acre_feet['Unit_of Good_Water_Consumption_per_Good_Per_Agent_0'])+(Maximum_Water_Consumption_Y_j[0]-np.dot(Production_Variables[0],Amount_Water_acre_feet['Unit_of Good_Water_Consumption_per_Good_Per_Agent_0']))*v
#F_2=np.dot(Production_Variables[1],Profit_Agent['Profit_Agent_1'])-Cost_Extracting_q_j[1]*np.dot(Production_Variables[1],Amount_Water_acre_feet['Unit_of Good_Water_Consumption_per_Good_Per_Agent_1'])+(Maximum_Water_Consumption_Y_j[1]-np.dot(Production_Variables[1],Amount_Water_acre_feet['Unit_of Good_Water_Consumption_per_Good_Per_Agent_1']))*v
#F_3=np.dot(Production_Variables[2],Profit_Agent['Profit_Agent_2'])-Cost_Extracting_q_j[2]*np.dot(Production_Variables[2],Amount_Water_acre_feet['Unit_of Good_Water_Consumption_per_Good_Per_Agent_2'])+(Maximum_Water_Consumption_Y_j[2]-np.dot(Production_Variables[2],Amount_Water_acre_feet['Unit_of Good_Water_Consumption_per_Good_Per_Agent_2']))*v

In [ ]:
#constraint_Universal_1=[Production_Variables[i][j]>=0 for i in range(Number_Agents) for j in range(len(Production_Variables[i]))]
#constraint_Universal_1

In [ ]:
#Production_Limits_of_each_Good=[get_array_of_size_k(i) for i in Types_Products]
#Production_Limits_of_each_Good

In [ ]:
#Sum_Upper_Bound=np.sum([Production_Limits_of_each_Good[i][j] for i in range(Number_Agents) for j in range(len(Production_Limits_of_each_Good[i]))])
#Sum_Upper_Bound

In [ ]:
#constraint_Universal_2=[Production_Variables[i][j]<=Production_Limits_of_each_Good[i][j] for i in range(0, Number_Agents-2) for j in range(len(Production_Variables[i]))]
#Constraint_Universal_3=[np.dot(Production_Variables[i], Amount_Water_acre_feet[Keysdict_Amount_Water_acre_feet[i]]) for i in range(Number_Agents)]
#Constraint_Universal_3_Final=[np.sum(Constraint_Universal_3)==Sum_Upper_Bound]
#All_Constraint=np.append(constraint_Universal_1, constraint_Universal_2)
#All_Constraint_Final=np.append(All_Constraint, np.array(Constraint_Universal_3_Final))
#All_Constraint_Final
#obj1= np.dot(Production_Variables[0],Profit_Agent[keysdict_Profit_Agent[0]])-Cost_Extracting_q_j[0]*np.dot(Production_Variables[0], Amount_Water_acre_feet[Keysdict_Amount_Water_acre_feet[0]])+(float(Maximum_Water_Consumption_Y_j[0])-np.dot(Production_Variables[0], Amount_Water_acre_feet[Keysdict_Amount_Water_acre_feet[0]]))*v
#obj2= np.dot(Production_Variables[0],Profit_Agent[keysdict_Profit_Agent[0]])-Cost_Extracting_q_j[0]*np.dot(Production_Variables[0], Amount_Water_acre_feet[Keysdict_Amount_Water_acre_feet[0]])+(float(Maximum_Water_Consumption_Y_j[1])-np.dot(Production_Variables[0], Amount_Water_acre_feet[Keysdict_Amount_Water_acre_feet[0]]))*v
#obj3=np.dot(Production_Variables[0],Profit_Agent[keysdict_Profit_Agent[0]])-Cost_Extracting_q_j[0]*np.dot(Production_Variables[0], Amount_Water_acre_feet[Keysdict_Amount_Water_acre_feet[0]])+(float(Maximum_Water_Consumption_Y_j[2])-np.dot(Production_Variables[0], Amount_Water_acre_feet[Keysdict_Amount_Water_acre_feet[0]]))*v
#w1=1/10
#w2=2/10
#Omega=np.array([w1, w2, 1-w1-w2])
#Objective=np.dot(Omega, [obj1, obj2, obj3])
#print(Omega)
#Objective_Total=cp.Problem(cp.Maximize(Objective), All_Constraint_Final)
#print("Problem Objective is DCP:", Objective_Total.is_dcp())
#Answer=Objective_Total.solve()
#print(Answer)

In [ ]:
#Constraint_Universal_3=[np.dot(Production_Variables[i], Amount_Water_acre_feet[Keysdict_Amount_Water_acre_feet[i]]) for i in range(Number_Agents)]
#Constraint_Universal_3_Final=[np.sum(Constraint_Universal_3)==Sum_Upper_Bound]

In [ ]:
#All_Constraint=np.append(constraint_Universal_1, constraint_Universal_2)
#All_Constraint_Final=np.append(All_Constraint, np.array(Constraint_Universal_3_Final))
#All_Constraint_Final

In [ ]:
#Constraint_Universal_3=[ np.dot(Production_Variables[0], Amount_Water_acre_feet['Unit_of Good_Water_Consumption_per_Good_Per_Agent_0']) + np.dot(Production_Variables[1], Amount_Water_acre_feet['Unit_of Good_Water_Consumption_per_Good_Per_Agent_1'])+np.dot(Production_Variables[2], Amount_Water_acre_feet['Unit_of Good_Water_Consumption_per_Good_Per_Agent_2'])==Maximum_Water_Consumption_Y_j[0]+Maximum_Water_Consumption_Y_j[1]+ Maximum_Water_Consumption_Y_j[2]]

In [ ]:

#constraint=[Production_Variables[0][0]>=0, Production_Variables[0][0]<=100000, Production_Variables[1][0]>=0, Production_Variables[1][0]<=100000, Production_Variables[2][0]>=0, Production_Variables[2][0]<=100000, Production_Variables[2][1]>=0, Production_Variables[2][1]<=100000, np.dot(Production_Variables[0], Amount_Water_acre_feet['Unit_of Good_Water_Consumption_per_Good_Per_Agent_0']) + np.dot(Production_Variables[1], Amount_Water_acre_feet['Unit_of Good_Water_Consumption_per_Good_Per_Agent_1'])+np.dot(Production_Variables[2], Amount_Water_acre_feet['Unit_of Good_Water_Consumption_per_Good_Per_Agent_2'])==Maximum_Water_Consumption_Y_j[0]+Maximum_Water_Consumption_Y_j[1]+ Maximum_Water_Consumption_Y_j[2]]

In [ ]:
#np.dot(Production_Variables[0],Amount_Water_acre_feet['Unit_of Good_Water_Consumption_per_Good_Per_Agent_0'])+
#np.dot(Production_Variables[1],Amount_Water_acre_feet['Unit_of Good_Water_Consumption_per_Good_Per_Agent_1'])+
#np.dot(Production_Variables[2],Amount_Water_acre_feet['Unit_of Good_Water_Consumption_per_Good_Per_Agent_2'])==Maximum_Water_Consumption_Y_j[0]+
#Maximum_Water_Consumption_Y_j[1]+Maximum_Water_Consumption_Y_j[2]

In [ ]:
#from numbers import Number
#Vector of size the number of agents minus 1 (n-1)
#Omega_Prime=[random.random() for i in range(Number_Agents-1)]
#Omega=np.append(Omega_Prime, np.array([1-np.sum(Omega_Prime)]))
#Omega

In [ ]:
#obj1= np.dot(Production_Variables[0],Profit_Agent[keysdict_Profit_Agent[0]])-Cost_Extracting_q_j[0]*np.dot(Production_Variables[0], Amount_Water_acre_feet[Keysdict_Amount_Water_acre_feet[0]])+(float(Maximum_Water_Consumption_Y_j[0])-np.dot(Production_Variables[0], Amount_Water_acre_feet[Keysdict_Amount_Water_acre_feet[0]]))*v
#obj2= np.dot(Production_Variables[0],Profit_Agent[keysdict_Profit_Agent[0]])-Cost_Extracting_q_j[0]*np.dot(Production_Variables[0], Amount_Water_acre_feet[Keysdict_Amount_Water_acre_feet[0]])+(float(Maximum_Water_Consumption_Y_j[1])-np.dot(Production_Variables[0], Amount_Water_acre_feet[Keysdict_Amount_Water_acre_feet[0]]))*v
#obj3=np.dot(Production_Variables[0],Profit_Agent[keysdict_Profit_Agent[0]])-Cost_Extracting_q_j[0]*np.dot(Production_Variables[0], Amount_Water_acre_feet[Keysdict_Amount_Water_acre_feet[0]])+(float(Maximum_Water_Consumption_Y_j[2])-np.dot(Production_Variables[0], Amount_Water_acre_feet[Keysdict_Amount_Water_acre_feet[0]]))*v

In [ ]:
#w1=random.random()
#w2=random.random()
#Omega=np.array([w1, w2, 1-w1-w2])
#obj=np.dot(Omega, [cp.log(item) for item in List4])
#obj=np.dot(Omega, cp.log(np.array(list(Dict_of_Functions.values()))))
#obj1=np.dot(Production_Variables[0],Profit_Agent[keysdict_Profit_Agent[0]])-Cost_Extracting_q_j[0]*np.dot(Production_Variables[0], Amount_Water_acre_feet[Keysdict_Amount_Water_acre_feet[0]])+(float(Maximum_Water_Consumption_Y_j[0])-np.dot(Production_Variables[0], Amount_Water_acre_feet[Keysdict_Amount_Water_acre_feet[0]]))*v
#obj= np.dot(Production_Variables[0],Profit_Agent[keysdict_Profit_Agent[0]])-Cost_Extracting_q_j[0]*np.dot(Production_Variables[0], Amount_Water_acre_feet[Keysdict_Amount_Water_acre_feet[0]])+(float(Maximum_Water_Consumption_Y_j[0])-np.dot(Production_Variables[0], Amount_Water_acre_feet[Keysdict_Amount_Water_acre_feet[0]]))*v
#+ np.dot(Production_Variables[0],Profit_Agent[keysdict_Profit_Agent[0]])-Cost_Extracting_q_j[0]*np.dot(Production_Variables[0], Amount_Water_acre_feet[Keysdict_Amount_Water_acre_feet[0]])+(float(Maximum_Water_Consumption_Y_j[1])-np.dot(Production_Variables[0], Amount_Water_acre_feet[Keysdict_Amount_Water_acre_feet[0]]))*v
#+np.dot(Production_Variables[0],Profit_Agent[keysdict_Profit_Agent[0]])-Cost_Extracting_q_j[0]*np.dot(Production_Variables[0], Amount_Water_acre_feet[Keysdict_Amount_Water_acre_feet[0]])+(float(Maximum_Water_Consumption_Y_j[2])-np.dot(Production_Variables[0], Amount_Water_acre_feet[Keysdict_Amount_Water_acre_feet[0]]))*v

In [ ]:
#Omega=np.array([1/10, 8/10, 1/10])
#w1=1/3
#w2=1/3
#Omega=np.array([w1, w2, 1-w1-w2])
#Objective=np.dot(Omega, [obj1, obj2, obj3])
#print(Omega)

In [ ]:
#All_Constraint

In [ ]:
#Objective_Total=cp.Problem(cp.Maximize(Objective), All_Constraint_Final)

In [ ]:
#print("Problem Objective is DCP:", Objective_Total.is_dcp())

In [ ]:
#Answer=Objective_Total.solve()
#print(Answer)

In [ ]:
#for i in range(Number_Agents):
 # for j in range(Types_Products[i]):
  #  print(Production_Variables[i][j].value)